In [1]:
import arxiv

client = arxiv.Client()
topic = 'llm'
search = arxiv.Search(
    query= topic,
    max_results=5,
    sort_by= arxiv.SortCriterion.Relevance
)
papers = client.results(search=search)

for paper in papers:
    print(paper)

http://arxiv.org/abs/2412.18022v1
http://arxiv.org/abs/2406.10300v1
http://arxiv.org/abs/2405.19888v1
http://arxiv.org/abs/2311.10372v2
http://arxiv.org/abs/2411.15764v1


In [2]:
import os, json
from typing import List
PAPER_DIR = "./papers"
def search_papers(topic: str, max_results: int = 5) -> List[str]:
    """
    주제를 기반으로 arXiv에서 논문을 검색하고 해당 정보를 저장한다.


    Args:
        topic: 검색할 주제
        max_results: 검색할 최대 결과 수 (기본값: 5)


    Returns:
        검색에서 찾은 논문 ID 목록
    """
    # arxiv를 사용하여 논문 찾기
    client = arxiv.Client()
    search = arxiv.Search(
        query     = topic,
        max_results = max_results,
        sort_by   = arxiv.SortCriterion.Relevance
    )
    papers = client.results(search)


    # 이 주제에 대한 디렉토리 생성
    path = os.path.join(PAPER_DIR, topic.lower().replace(" ", "_"))
    os.makedirs(path, exist_ok=True)


    file_path = os.path.join(path, "papers_info.json")


    # 기존 논문 정보 로드 시도
    try:
        with open(file_path, "r") as json_file:
            papers_info = json.load(json_file)
    except (FileNotFoundError, json.JSONDecodeError):
        papers_info = {}


    # 각 논문을 처리하고 papers_info에 추가
    paper_ids = []
    for paper in papers:
        paper_id   = paper.get_short_id()
        paper_info = {
            "title"    : paper.title,
            "authors"  : [author.name for author in paper.authors],
            "summary"  : paper.summary,
            "pdf_url"  : paper.pdf_url,
            "published": str(paper.published.date())
        }
        paper_ids.append(paper_id)
        papers_info[paper_id] = paper_info


    # 업데이트된 papers_info를 json 파일에 저장
    with open(file_path, "w") as json_file:
        json.dump(papers_info, json_file, indent=2)


    print(f"결과가 다음 위치에 저장되었습니다: {file_path}")
    return paper_ids


In [3]:
def extract_info(paper_id: str) -> str:
    """
    모든 주제 디렉토리에서 특정 논문에 관한 정보를 검색한다.
   
    Args:
        paper_id: 검색할 논문의 ID
       
    Returns:
        찾은 경우 논문 정보가 담긴 JSON 문자열, 찾지 못한 경우 오류 메시지
    """
 
    for item in os.listdir(PAPER_DIR):
        item_path = os.path.join(PAPER_DIR, item)
        if os.path.isdir(item_path):
            file_path = os.path.join(item_path, "papers_info.json")
            if os.path.isfile(file_path):
                try:
                    with open(file_path, "r") as json_file:
                        papers_info = json.load(json_file)
                        if paper_id in papers_info:
                            return json.dumps(papers_info[paper_id], indent=2)
                except (FileNotFoundError, json.JSONDecodeError) as e:
                    print(f"{file_path} 읽기 오류: {str(e)}")
                    continue
   
    return f"논문 {paper_id}와 관련된 저장된 정보가 없다."

In [4]:
search_papers('llm', 5)

결과가 다음 위치에 저장되었습니다: ./papers/llm/papers_info.json


['2412.18022v1',
 '2406.10300v1',
 '2405.19888v1',
 '2311.10372v2',
 '2411.15764v1']

In [5]:
extract_info('2412.18022v1')

'{\n  "title": "Trustworthy and Efficient LLMs Meet Databases",\n  "authors": [\n    "Kyoungmin Kim",\n    "Anastasia Ailamaki"\n  ],\n  "summary": "In the rapidly evolving AI era with large language models (LLMs) at the core,\\nmaking LLMs more trustworthy and efficient, especially in output generation\\n(inference), has gained significant attention. This is to reduce plausible but\\nfaulty LLM outputs (a.k.a hallucinations) and meet the highly increased\\ninference demands. This tutorial explores such efforts and makes them\\ntransparent to the database community. Understanding these efforts is essential\\nin harnessing LLMs in database tasks and adapting database techniques to LLMs.\\nFurthermore, we delve into the synergy between LLMs and databases, highlighting\\nnew opportunities and challenges in their intersection. This tutorial aims to\\nshare with database researchers and practitioners essential concepts and\\nstrategies around LLMs, reduce the unfamiliarity of LLMs, and insp

In [6]:
search_papers('Model Compression using Sparse Matrices', 10)

결과가 다음 위치에 저장되었습니다: ./papers/model_compression_using_sparse_matrices/papers_info.json


['2105.11025v1',
 '2102.07071v1',
 '2210.11420v1',
 '2506.04208v1',
 '1905.07931v1',
 '1207.2079v1',
 '1312.0525v2',
 '2409.08699v2',
 '1707.08208v1',
 '2303.16106v1']

In [7]:
extract_info('2105.11025v1')

'{\n  "title": "Compressing Heavy-Tailed Weight Matrices for Non-Vacuous Generalization Bounds",\n  "authors": [\n    "John Y. Shin"\n  ],\n  "summary": "Heavy-tailed distributions have been studied in statistics, random matrix\\ntheory, physics, and econometrics as models of correlated systems, among other\\ndomains. Further, heavy-tail distributed eigenvalues of the covariance matrix\\nof the weight matrices in neural networks have been shown to empirically\\ncorrelate with test set accuracy in several works (e.g. arXiv:1901.08276), but\\na formal relationship between heavy-tail distributed parameters and\\ngeneralization bounds was yet to be demonstrated. In this work, the compression\\nframework of arXiv:1802.05296 is utilized to show that matrices with heavy-tail\\ndistributed matrix elements can be compressed, resulting in networks with\\nsparse weight matrices. Since the parameter count has been reduced to a sum of\\nthe non-zero elements of sparse matrices, the compression fram

In [8]:
functions = [
    {
        "name": "search_papers",
        "description": "주제를 기반으로 arXiv에서 논문을 검색하고 해당 정보를 저장한다.",
        "parameters": {
            "type": "object",
            "properties": {
                "topic": {
                    "type": "string",
                    "description": "검색할 주제"
                },
                "max_results": {
                    "type": "integer",
                    "description": "검색할 최대 결과 수",
                    "default": 5
                }
            },
            "required": ["topic"]
        }
    },
    {
        "name": "extract_info",
        "description": "모든 주제 디렉토리에서 특정 논문에 관한 정보를 검색한다.",
        "parameters": {
            "type": "object",
            "properties": {
                "paper_id": {
                    "type": "string",
                    "description": "검색할 논문의 ID"
                }
            },
            "required": ["paper_id"]
        }
    }
]


mapping_tool_function  = {
    "search_papers" : search_papers,
    "extract_info"  : extract_info
}


def execute_tool(tool_name, tool_args):
    result = mapping_tool_function[tool_name](**tool_args)


    if result is None:
        return "작업이 완료되었찌만 결과는 반환하지 않았다"
   
    if isinstance(result, list):
        return ", ".join(result)
   
    if isinstance(result, dict):
        return json.dumps(result, indent=2, ensure_ascii=False)
   
    return str(result)

In [9]:
from openai import OpenAI
client = OpenAI()

messages = [{'role' : 'user', 'content' : "llm에 대해서 알려줘"}]
resp  = client.chat.completions.create(
    model='gpt-4-0125-preview',
    messages=messages,
    functions=functions,
    function_call='auto',
    max_tokens=2024
)

In [ ]:
resp.choices[0].message

msg = resp.choices[0].message
call = msg.function_call

In [ ]:
tool_result = execute_tool(call.name,json.loads(call.arguments) )

In [ ]:
def process_query(query: str):
    messages = [{"role": "user", "content": query}]

    while True:
        resp = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            functions=functions,
            function_call="auto",
            max_tokens=2024
        )
        msg = resp.choices[0].message

        # 1) 일반 텍스트 응답
        if msg.content is not None:
            print(msg.content)
            break

        # 2) 함수 호출 응답
        call = msg.function_call
        fname = call.name
        fargs = json.loads(call.arguments)

        print(f"Calling tool `{fname}` with args: {fargs}")
        tool_result = execute_tool(fname, fargs)

        # assistant의 함수 호출 메시지를 기록
        messages.append({
            "role": "assistant",
            "function_call": {
                "name": fname,
                "arguments": call.arguments
            }
        })
        # 함수 실행 결과를 function 역할로 추가
        messages.append({
            "role": "function",
            "name": fname,
            "content": tool_result
        })

In [ ]:
def chat_loop():
    print("쿼리를 입력하거나 'quit'를 입력해 종료합니다.")
    while True:
        query = input("Query: ").strip()
        if query.lower() == "quit":
            break
        try:
            process_query(query)
       
        except Exception as e:
            print(f"Error: {e}")